<a href="https://colab.research.google.com/github/Trung0Minh/AIO2023-MODULE-5/blob/main/object_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import pickle
import numpy as np

def getFaceLocation():
    camera = cv2.VideoCapture(0)
    facecascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    ret = True

    # Refresh the camera
    for i in range(5):
        ret, frame = camera.read()
    while ret:
        ret, frame = camera.read()
        if ret:
            # Convert the frame to grayscale for face detection
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # Detect faces using Haar Cascade Classifier
            faces = facecascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            # If at least one face is detected, process the first face
            if len(faces) > 0:
                (x, y, w, h) = faces[0]  # Take the first detected face
                face_image = frame[y:y+h, x:x+w]  # Extract the face region
                face_location = (x, y, w, h)  # Store the location as (x, y, width, height)
                # Release resources and return the face image and location
                camera.release()
                cv2.destroyAllWindows()
                return face_image, face_location
        else:
            print('error')
            break
    # If no face is detected or an error occurs, release resources and return None
    camera.release()
    cv2.destroyAllWindows()
    return None, None

face_image, face_location = getFaceLocation()

In [ ]:
import cv2
import numpy as np

def face_tracking(template, method):
    ret = True
    camera = cv2.VideoCapture(0)
    # Refresh the camera
    for i in range(5):
        ret, frame = camera.read()
    # Assume template is provided (e.g., from Problem 1); here we initialize with the first frame for demonstration
    ret, frame = camera.read()
    template = frame[100:200, 100:200]  # Example ROI; in practice, use face_image from Problem 1
    template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    method = cv2.TM_CCOEFF_NORMED  # Define the matching method

    while ret:
        ret, frame = camera.read()
        if ret:
            img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # Store width and height of template
            w, h = template.shape[::-1]
            res = cv2.matchTemplate(img_gray, template, method)
            # Find the best match location
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
            # For TM_CCOEFF_NORMED, use max_loc for the best match
            if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
                top_left = min_loc
            else:
                top_left = max_loc
            bottom_right = (top_left[0] + w, top_left[1] + h)
            # Draw rectangle around the matched region
            cv2.rectangle(frame, top_left, bottom_right, 255, 2)
            cv2.imshow('template', template)
            cv2.imshow('frames', frame)
            # Update template with the matched region, converting to grayscale
            cropped = frame[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
            template = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
            if cv2.waitKey(1) == 27:  # Exit on ESC key
                break
    camera.release()
    cv2.destroyAllWindows()  # Replace close_window_os() with standard OpenCV function

In [ ]:
import numpy as np
import cv2 as cv
import argparse
from google.colab.patches import cv2_imshow

def object_tracking_video_file():
    cap = cv.VideoCapture("video.mp4")  # Replace with actual video path
    # Take first frame of the video
    ret, frame = cap.read()
    # Setup initial location of window
    x, y, w, h = (300, 400, 200, 200)  # Hardcoded values
    track_window = (x, y, w, h)
    # Set up the ROI for tracking
    roi = frame[y:y+h, x:x+w]
    # Compute histogram and setup for Mean Shift
    hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
    roi_hist = cv.calcHist([hsv_roi], [0], None, [180], [0, 180])
    cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)
    term_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)

    history = []
    while True:
        ret, frame = cap.read()
        if ret:
            hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
            dst = cv.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
            # Apply Mean Shift to get the new location
            ret, track_window = cv.meanShift(dst, track_window, term_crit)
            # Draw it on image
            x, y, w, h = track_window
            img2 = cv.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 0), 4)
            history.append((x + int(w/2), y + int(h/2)))
            if len(history) >= 2:
                for i in range(len(history) - 1):
                    (x1, y1) = history[i]
                    (x2, y2) = history[i + 1]
                    cv.line(img2, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2_imshow(img2)
            k = cv.waitKey(30) & 0xff
            if k == 27:
                break
        else:
            break
    cap.release()
    cv.destroyAllWindows()

object_tracking_video_file()